In [1]:
! pip install smolagents ddgs langdetect langchain_community rank_bm25 "smolagets[litellm]" plotly geopandas shepely kaleido -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement shepely (from versions: none)
ERROR: No matching distribution found for shepely


In [2]:
from huggingface_hub import notebook_login
notebook_login()   

In [4]:
import math

from smolagents import CodeAgent, ToolCallingAgent, DuckDuckGoSearchTool, InferenceClientModel, tool, Tool
from langdetect import detect
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from typing import Optional, Tuple

In [5]:
model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"

# CodeAgent

## Web Search

In [11]:
agent = CodeAgent(
    tools=[
    DuckDuckGoSearchTool()
    ],
    model=InferenceClientModel(model_id=model_name)
    # choosing a coder model to prevent errors from CodeAgent
)

In [ ]:
agent.run("Search for the best music recommendations for a Minecraft gaming night")

## Custom Tool

In [14]:
# Tool to suggest a Minecraft minigame based on your personal preferances

@tool
def suggest_minigame(num_player: int, pvp_allowed: bool, game_type: str) -> str:
    """
    Suggests a Minecraft minigame based on number of players and PvP choice.
    Args:
        num_player (int): Number of players 
        pvp_allowed (bool): Should the game contain PvP?
        game_type: (str): The type of game. Allowed values are:
                        - Survival
                        - Creative
    """

    if num_player <= 1:
        
        if pvp_allowed == True:
            if game_type == "Survival":
                return "Hunger Games"
            elif game_type == "Creative":
                return "Temple Lockout"
                
        elif pvp_allowed == False:
            if game_type == "Survival":
                return "Spleef"
            elif game_type == "Creative":
                return "Build Battle"

    if num_player >= 2 and num_player <= 4:

        if pvp_allowed == True:
            if game_type == "Survival":
                return "Bed Wars"
            if game_type == "Creative":
                return "Sky Block"

        elif pvp_allowed == False:
            if game_type == "Survival":
                return "Hyper-Drive"
            if game_type == "Creative":
                return "Order Up"
                
    else:
        return "Game not found!"

In [17]:
agent = CodeAgent(
    tools=[suggest_minigame],
    model=InferenceClientModel(model_id=model_name)
)

In [ ]:
agent.run("Choose a minecraft survival minigame for 3 players where combat is not prohibited.")

## Custom Tool with Python Import Inside the Agent

In [ ]:
# Tool to detect the server's primary language based on it's name

agent = CodeAgent(
    tools=[],
    model=InferenceClientModel(model_id=model_name),
    additional_authorized_imports=["langdetect"]
)

In [ ]:
server_name = "Servidor Cobblemon"

agent.run(
    f"""
    I need to find a Minecraft server that has English instructions. Here is the server name:
    {server_name}
    Determine it's language and tell me if I should play in this server.
    """
)

In [28]:
# TODO: Inspecting the Agent with OpenTelemetry and Langfuse

# ToolCallingAgent

In [30]:
agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=InferenceClientModel(model_id=model_name),
)

In [ ]:
agent.run("Search for the best music recommendations for a Minecraft gaming night")

# Tools

In [17]:
# defining a tool with '@tool' decorator

@tool
def pc_suggestion_tool(budget: float) -> str:
    """
    This tool suggests a gaming computer based on your budget.
    It uses a special formula to compute price-performance ratio:
    (GPU_score * memory / price)
    and returns the max value that is within your price range.

    Args:
        budget: User's budget in USD ($)
    
    """
    all_comps = [
        {"name": "MSI Codex R2", "GPU_score": 4060, "memory": 16, "price": 1120},
        {"name": "HP Omen 35L", "GPU_score": 5060, "memory": 8, "price": 1080},
        {"name": "Alienware Area-51", "GPU_score": 5070, "memory": 12, "price": 2950},
    ]

    suitable_comps = [comp for comp in all_comps if comp["price"] <= budget]
    
    scores = {}
    for comp in suitable_comps:
        name, gpu, memory, price = comp["name"], comp["GPU_score"], comp["memory"], comp["price"]
        score = (gpu * memory) / price
        scores[name] = score

    best_comp = max(scores, key=scores.get)
    return best_comp
    

In [ ]:
agent = CodeAgent(
    tools=[pc_suggestion_tool], 
    model=InferenceClientModel(model_id=model_name)
)

agent.run(
    "Can you give me the best PC for a budget of 1200$"
)

In [46]:
# defining a Tool as a Python Class

class MinecraftSkinTool(Tool):
    name = "minecraft_skin_finder_tool"
    description = """
    This tool suggests a Minecraft skin based on your choice of a character.
    It returns a URL string from 'https://www.minecraftskins.com/'
    """

    inputs = {
        "character": {
            "type": "string",
            "description": "The character (e.g., 'Batman', 'Spiderman', 'Yoshi', 'Cool Duck', 'Don Corleone')"
        }
    }

    output_type = "string"

    def forward(self, character: str):
        skin_urls = {
            "Batman": "https://www.minecraftskins.com/skin/23455341/batman---comic-style/",
            "Spiderman": "https://www.minecraftskins.com/skin/23467100/spider-man--brand-new-day-/",
            "Yoshi": "https://www.minecraftskins.com/skin/23480391/yoshi-derp/",
            "Cool Duck": "https://www.minecraftskins.com/skin/23383526/cool-duck-/",
            "Don Corleone": "https://www.minecraftskins.com/skin/23447255/-don-corleone-/"
        }

        return skin_urls.get(character.lower(), 
                            "Skin not found! Try building your own at: https://www.minecraftskins.com/skin-editor/")

In [ ]:
skin_finder_tool = MinecraftSkinTool()
agent = CodeAgent(
    tools=[skin_finder_tool],
    model=InferenceClientModel(model_id=model_name)
)

agent.run(
    "I want a Minecraft skin with Yoshi."
) # try with 'the dinosaur from Mario'

# Agentic Retrieval

In [ ]:
# basic retrieval with DuckDuckGo

search_tool = DuckDuckGoSearchTool()

model = InferenceClientModel(model_id=model_name)

agent = CodeAgent(
    tools=[search_tool],
    model=model
)

agent.run(
    "Search for best gaming PCs under $1200."
)

In [10]:
# custom knowledge base tool

class BuildIdeasRetrieverTool(Tool):
    name = "build_ideas_retriever"
    description = "Uses semantic search to retrieve relevant build ideas for Minecraft."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be a theme."
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n" + "".join(
            [
                f"\n\n===== Idea {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)
            ]
        )

In [15]:
# simulate a knowledge base about Minecraft ideas

build_ideas = [
    {"text": "Huge Cottage with a flower garden: Warm and cozy cottage with a beautiful flower garden and gentle willow trees. There’s a little greenhouse at the front.", "source": "Build Ideas"},
    {"text": "Simple mediterranean villa for Plains Biome: Mediterranean-style house with a cozy tower and a lovely yard! There’s even a big orange tree right beside it. It’s a great fit for a plains or desert biome, and the interior is made for two players!", "source": "Build Ideas"}, 
    {"text": "Factions: A complicated game with many different aspects, so it's important to understand how to play it before you start. The basic explanation goes as such. You claim land, get money, build, declare war, fight enemies, and don't die.", "source": "Minigame Ideas"},
    {"text": "Small Bee House: Small bee apiary with a greenhouse and a garden that is perfect for your cottagecore bee farm! There is even a place to make honey and wax your copper!", "source": "Build Ideas"},
    {"text": "Spleef: Spleef is a competitive minigame played within Minecraft. When playing spleef, players destroy blocks below other players, allowing them to fall off the playing field and into a pit. The object of the game is to be the last player on the field.", "source": "Minigame Ideas"}
]   # docs may need refining

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in build_ideas
]

# split the docs into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

In [ ]:
build_ideas_retriever = BuildIdeasRetrieverTool(docs_processed)

agent = CodeAgent(
    tools=[build_ideas_retriever],
    model=InferenceClientModel(model_id=model_name)
)

agent.run("Find me some Minecraft survival games.")

# Multi-Agent Systems